In [1]:
import polars as pl

In [7]:
pl.read_parquet('dude_data/blocks.parquet')

block_id,block,timestamp,n_txs,is_mainnet
str,i64,"datetime[μs, UTC]",i64,bool
"""00000000000000000001a80ff6a5bb…",896334,2025-05-12 00:40:59 UTC,2672,true
"""000000000000000000020f4e0c2902…",896335,2025-05-12 01:14:19 UTC,3198,true
